In [1]:
import pandas as pd
import numpy as np
import nltk
import lxml.html
import re
import string
import gensim
import matplotlib.pyplot as plt 
from textacy import preprocessing
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import os
import nltk

from nltk.corpus import stopwords
# nltk.download('stopwords')  
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
import swifter
# Importing Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

2023-04-10 12:36:52.782257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 12:36:53.711143: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-10 12:36:53.711257: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-10 12:36:53.711269: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# df = pd.read_csv(r"/home/ubuntu/Downloads/text_to_cluster.txt", sep = "\t", header=None,  names=["texts"])

In [3]:
texts_filepath = r"/home/ubuntu/Downloads/text_to_cluster.txt"

with open(texts_filepath) as infile:
    data = infile.readlines()
    
texts_df = pd.DataFrame(data={"texts": data}, columns=["texts"])
texts_df.head()

,texts
0,Ransomware attack at Mexico's Pemex halts work...
1,#city | #ransomware | Ransomware Attack At Mex...
2,"Mexico's Pemex Oil Suffers Ransomware Attack, ..."
3,A Mexican oil company was hit by ransomware at...
4,Pemex Struck by Ransomware Attack\n


In [4]:
texts_df.shape, texts_df.size

((684, 1), 684)

In [5]:
texts_df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
679     True
680     True
681     True
682     True
683     True
Length: 684, dtype: bool

In [6]:
# texts_df.drop_duplicates(inplace = True)

In [7]:
texts_df

,texts
0,Ransomware attack at Mexico's Pemex halts work...
1,#city | #ransomware | Ransomware Attack At Mex...
2,"Mexico's Pemex Oil Suffers Ransomware Attack, ..."
3,A Mexican oil company was hit by ransomware at...
4,Pemex Struck by Ransomware Attack\n
...,...
679,Detecting and Responding to Ransomware\n
680,"Emotet malware is back, more dangerous than ev..."
681,Hosting provider SmarterASP.NET hit by ransomw...
682,Ransomware as a Service (RaaS) – A Contemporar...


## Text Preprocessing

In [8]:
class text_preprocess:
    
    def __init__(self):
        pass

    def convert_to_lower(self, text):
        return text.lower()

    def remove_emojis(self, text):
        text = re.sub(r"(?:\@|https?\://)\S+", " ", text) #remove links and mentions
        text = re.sub(r"<.*?>"," ",text)

        wierd_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emotions
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u'\U00010000-\U0010ffff'
            u"\u200d"
            u"\u2640-\u2642"
            u"\u2600-\u2B55"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\u3030"
            u"\ufe0f"
            u"\u2069"
            u"\u2066"
            u"\u200c"
            u"\u2068"
            u"\u2067"
            "]+", flags=re.UNICODE)

        rm_emoji = wierd_pattern.sub(r'', text)
        return rm_emoji

    def remove_html(self, text):
        html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        rm_html = re.sub(html, ' ', text)
        return rm_html

    def remove_URL(self,text):
        url = re.compile(r'https?://\S+|www\.\S+')
        URL = url.sub(r' ', text)
        return URL
    
    def remove_non_ascii(self, text):
        return re.sub(r'[^\x00-\x7f]',r' ', text) # or ''.join([x for x in text if x in string.printable]) 
    
    
    def remove_numbers(self, text):
        number_pattern = r'\d+'
        without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
        return without_number


    def remove_punctuation(self,text):
        return text.translate(str.maketrans('', '', string.punctuation))
    
    def remove_keywords(self, text):
        return re.sub("\n|\r", "", text)


    def remove_extra_white_spaces(self, text):
        single_char_pattern = re.compile(r'\s+[a-zA-Z]\s+')
        without_sc = re.sub(single_char_pattern, r" ", text)
#         without_sc = text.replace(' ', '')
        return without_sc


    def preprocessText(self,text):            
        return self.remove_extra_white_spaces(self.remove_non_ascii(self.remove_URL(self.remove_html(self.remove_punctuation(self.remove_numbers(self.remove_emojis(self.remove_keywords(self.convert_to_lower(text)))))))))


In [9]:
if __name__ == "__main__":    
    text_prpocess_obj = text_preprocess()
    texts_df['cleaned_texts'] = texts_df.texts.swifter.apply(lambda x: text_prpocess_obj.preprocessText(x))

Pandas Apply:   0%|          | 0/684 [00:00<?, ?it/s]

In [10]:
texts_df.cleaned_texts

0      ransomware attack at mexicos pemex halts work ...
1      city  ransomware  ransomware attack at mexico ...
2      mexicos pemex oil suffers ransomware attack   ...
3      a mexican oil company was hit by ransomware at...
4                      pemex struck by ransomware attack
                             ...                        
679               detecting and responding to ransomware
680      emotet malware is back more dangerous than ever
681    hosting provider smarteraspnet hit by ransomwa...
682    ransomware as service raas contemporary mal du...
683    ransomware as service raas contemporary mal du...
Name: cleaned_texts, Length: 684, dtype: object

## Tokenization

In [11]:
from typing import List, Dict
from nltk import word_tokenize
from nltk.corpus import stopwords
# from spacy.lang.en.stop_words import STOP_WORDS
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
texts_df

,texts,cleaned_texts
0,Ransomware attack at Mexico's Pemex halts work...,ransomware attack at mexicos pemex halts work ...
1,#city | #ransomware | Ransomware Attack At Mex...,city ransomware ransomware attack at mexico ...
2,"Mexico's Pemex Oil Suffers Ransomware Attack, ...",mexicos pemex oil suffers ransomware attack ...
3,A Mexican oil company was hit by ransomware at...,a mexican oil company was hit by ransomware at...
4,Pemex Struck by Ransomware Attack\n,pemex struck by ransomware attack
...,...,...
679,Detecting and Responding to Ransomware\n,detecting and responding to ransomware
680,"Emotet malware is back, more dangerous than ev...",emotet malware is back more dangerous than ever
681,Hosting provider SmarterASP.NET hit by ransomw...,hosting provider smarteraspnet hit by ransomwa...
682,Ransomware as a Service (RaaS) – A Contemporar...,ransomware as service raas contemporary mal du...


In [72]:
# texts_df.cleaned_texts.tolist()

In [15]:
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def Tokenize_text(text: str) -> List[str]:
    text_tokens = word_tokenize(text)
    text_without_stopwords = [word for word in text_tokens if word not in stop_words]
    return text_without_stopwords

In [17]:
texts_df['cleaned_texts'] = texts_df.cleaned_texts.swifter.apply(lambda x: Tokenize_text(x))

Pandas Apply:   0%|          | 0/684 [00:00<?, ?it/s]

In [18]:
texts_df.cleaned_texts.tolist()[0]

['ransomware',
 'attack',
 'mexicos',
 'pemex',
 'halts',
 'work',
 'threatens',
 'cripple',
 'computers']

## Word2vec

In [19]:
from gensim.models import Word2Vec

In [20]:
w2v_model = Word2Vec(sentences=texts_df['cleaned_texts'], vector_size=100, workers=2)

In [21]:
w2v_model.wv.most_similar("threatens")

[('ransomware', 0.6307824850082397),
 ('malware', 0.6269527673721313),
 ('wanted', 0.6132566332817078),
 ('hit', 0.5850839018821716),
 ('attack', 0.5803406238555908),
 ('undercuts', 0.5587689876556396),
 ('says', 0.5577048063278198),
 ('discount', 0.5457214117050171),
 ('ransomwareasaservice', 0.5430466532707214),
 ('threat', 0.5397765636444092)]

In [22]:
w2v_model.wv['threatens']

array([ 3.8781425e-03,  2.4851218e-03, -9.2181386e-03,  1.1497257e-02,
       -8.4486911e-03, -6.2804255e-03,  9.7399456e-03,  1.3583688e-02,
       -1.6752420e-02,  1.6264303e-03, -7.2079194e-03, -1.2795372e-02,
        7.5292001e-03, -4.6591870e-03, -6.8214643e-03, -4.8563215e-03,
        7.9989983e-03, -5.8800885e-03, -9.2406543e-03, -1.6581168e-02,
        9.4210869e-03,  1.1988385e-02,  1.2736007e-02, -3.3261017e-03,
        3.2098107e-03,  4.0364489e-03, -4.0354542e-03, -5.1953364e-05,
       -4.1297433e-04,  1.1548216e-03,  8.8481815e-04, -3.6712994e-03,
        1.1009533e-02, -1.1242375e-02,  9.6844451e-07,  1.0312642e-02,
       -4.9331214e-04, -8.3236955e-03,  1.1164651e-03, -1.4815392e-02,
       -9.0940185e-03, -7.1771266e-03,  5.4350505e-03,  7.0203436e-03,
        4.5460318e-03,  9.3611749e-03, -1.0652624e-02, -4.4086911e-03,
       -4.4717300e-03, -4.2748818e-04,  9.1260290e-03, -1.1159301e-02,
       -7.2361133e-03, -9.4107082e-03,  8.0551999e-03,  8.1603014e-04,
      

In [23]:
def create_vectors(docs: List[List[str]]) -> List[List[float]]:
    
    all_vectors: List = []

    for text_tokens in docs:
        zero_vector = np.zeros(w2v_model.vector_size)
        vectors: List = []
        for token in text_tokens:
            if token in w2v_model.wv:
                try:
                    vectors.append(w2v_model.wv[token])
                except KeyError:
                    continue
                    
        if vectors:
            vectors = np.asarray(vectors)
            avg_vector = vectors.mean(axis=0)
            all_vectors.append(avg_vector)
        else:
            all_vectors.append(zero_vector)
    return all_vectors

In [24]:
all_text_vectors = create_vectors(texts_df.cleaned_texts)

In [25]:
len(all_text_vectors)

684

In [26]:
all_text_vectors[0]

array([-4.31032153e-03,  2.41161603e-03, -1.24638798e-04,  7.17833312e-03,
        2.03378731e-04, -1.82368197e-02,  3.19226971e-03,  2.03472897e-02,
       -1.40013164e-02, -9.55930166e-03,  1.76584930e-03, -1.37192141e-02,
       -2.39620148e-03,  6.47950917e-03,  2.94731953e-03,  1.37045700e-03,
        6.49562059e-03, -9.55815241e-03, -7.51781045e-03, -1.78936291e-02,
        1.03589275e-03,  2.71309307e-03,  5.59775997e-03, -7.23774964e-03,
        6.20342442e-04,  3.22882598e-03, -1.15274526e-02, -5.54135442e-03,
       -4.53272974e-03,  1.44901162e-03,  3.84759577e-03,  2.70541734e-03,
        6.34612143e-03, -8.47783964e-03, -2.97076069e-03,  1.62423775e-02,
        4.75747744e-03, -5.17467456e-03, -7.03626487e-04, -1.39664756e-02,
       -4.38939547e-03, -5.52493799e-03, -2.60638684e-04,  2.15352885e-03,
        6.43387437e-03,  3.61297117e-03, -4.64133266e-03,  8.25864496e-04,
        3.90515430e-03,  9.80034750e-03,  2.73367763e-03, -4.15032078e-03,
        8.35490355e-04,  

In [27]:
len(all_text_vectors[0])

100

In [28]:
from sklearn.cluster import KMeans

In [37]:
for x in range(200, 300):
    kmeans_model = KMeans(n_clusters=x).fit(all_text_vectors)
    y_pred = kmeans_model.predict(all_text_vectors)
    print(f"silhouette_scoreof n_cluster{x}",silhouette_score(all_text_vectors, y_pred))

silhouette_scoreof n_cluster200 0.7338090158315508
silhouette_scoreof n_cluster201 0.7399178125821569
silhouette_scoreof n_cluster202 0.7372220599487314
silhouette_scoreof n_cluster203 0.7484955259963518
silhouette_scoreof n_cluster204 0.7485426471299598
silhouette_scoreof n_cluster205 0.7552913382227275
silhouette_scoreof n_cluster206 0.7538072524192903
silhouette_scoreof n_cluster207 0.741843735988947
silhouette_scoreof n_cluster208 0.7539198708891945
silhouette_scoreof n_cluster209 0.7491503084018083
silhouette_scoreof n_cluster210 0.7568340997606604
silhouette_scoreof n_cluster211 0.7557235293261145
silhouette_scoreof n_cluster212 0.7597479363357609
silhouette_scoreof n_cluster213 0.7587419174124738
silhouette_scoreof n_cluster214 0.7598910459718592
silhouette_scoreof n_cluster215 0.7671804519389621
silhouette_scoreof n_cluster216 0.767255257690673
silhouette_scoreof n_cluster217 0.7648683309990618
silhouette_scoreof n_cluster218 0.7722879689838491
silhouette_scoreof n_cluster219 0

In [38]:
kmeans_model = KMeans(n_clusters=274).fit(all_text_vectors)

In [39]:
texts_df["cluster_label"] = kmeans_model.labels_

In [73]:
texts_df.head()

,texts,cleaned_texts,cluster_label
0,Ransomware attack at Mexico's Pemex halts work...,"[ransomware, attack, mexicos, pemex, halts, wo...",131
1,#city | #ransomware | Ransomware Attack At Mex...,"[city, ransomware, ransomware, attack, mexico,...",260
2,"Mexico's Pemex Oil Suffers Ransomware Attack, ...","[mexicos, pemex, oil, suffers, ransomware, att...",248
3,A Mexican oil company was hit by ransomware at...,"[mexican, oil, company, hit, ransomware, attack]",4
4,Pemex Struck by Ransomware Attack\n,"[pemex, struck, ransomware, attack]",26


In [40]:
texts_df.cluster_label.value_counts()

2      14
0      14
90      7
3       7
16      7
       ..
223     1
238     1
199     1
211     1
245     1
Name: cluster_label, Length: 274, dtype: int64

In [41]:
kmeans_model.inertia_

0.00017464702094648939

In [42]:
y_pred = kmeans_model.predict(all_text_vectors)

In [43]:
y_pred

array([131, 260, 248,   4,  26, 235,  71,  20, 247, 178, 131, 255, 168,
       213, 135, 234,  53, 218, 107, 120, 115, 237, 210, 198, 129,  61,
       169, 154, 245, 104, 113, 118, 246,   2,  15,  67,  30,   2,   2,
       160,  47,  47,  90,  90,   2,  90, 143,  11, 270,  78, 126, 156,
         2, 197, 231, 215,  54, 177, 102, 208, 258,  89, 266, 261, 251,
        95, 264, 195,  31, 252, 257, 133, 252, 227, 133, 249,  51, 191,
       268, 243, 230, 204, 236, 172, 148,   1, 197,   2, 165,   7, 253,
       273,  82, 100,  35,  94,  49,  81,  49, 168,  30,   2,   2, 160,
        47,  47,   2,  90, 143,  11, 270,  78, 126, 156,   2,  95, 243,
       230, 204, 236, 172, 148,   1,   7, 100,  35,  94,  81, 232, 267,
        36, 140, 224,  34,  43,  55, 116, 233,  79, 189,   0,   6,  41,
        43,  40, 244,  72,   5,  46,  19, 101,  68,  65, 132,  12,  82,
       119,  37, 120, 200,  99,  77, 182, 182,  36, 140,  34,  55,   0,
        37, 120,  99,  77,  58,   0,  16,  14,  13,  13,  66,  6

In [44]:
from sklearn.metrics import silhouette_score

In [45]:
print("silhouette_score(n_cluster=20)",silhouette_score(all_text_vectors, y_pred))

silhouette_score(n_cluster=20) 0.8713735766533465


In [64]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics

In [65]:
db = DBSCAN(eps=0.02, min_samples=10).fit(all_text_vectors)
labels = db.labels_

In [66]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 5
Estimated number of noise points: 508


In [69]:
print(f"Silhouette Coefficient: {metrics.silhouette_score(all_text_vectors, labels):.3f}")

Silhouette Coefficient: -0.055


In [70]:
db.labels_

array([ 0,  0,  0,  0, -1, -1,  0,  0,  0,  0,  0,  0, -1,  1,  1,  1,  1,
        1, -1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  2,
        1, -1, -1,  2,  2, -1, -1, -1, -1, -1,  2, -1, -1, -1, -1, -1, -1,
       -1,  2,  3, -1,  3,  3, -1,  3,  3,  3,  3,  3,  3,  3, -1,  3, -1,
        3,  3,  3,  3,  3,  3,  3,  3, -1, -1,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  2, -1, -1,  3,  3, -1, -1, -1, -1, -1, -1, -1, -1, -1,  2,
        2, -1, -1, -1,  2, -1, -1, -1, -1, -1, -1, -1,  2, -1,  3,  3,  3,
        3,  3,  3,  3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  4, -1,
       -1, -1, -1, -1,  4, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1